# Predicting Material Backorder in Inventory Management

## App using Streamlit

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score, roc_curve, auc

In [12]:
#constants calculated from eda & feature engineering
lead_time_mean = float(np.load('lead_time_mean.npy'))
potential_issue_probability_matrix = pd.read_csv('potential_issue_probability_matrix.csv')
deck_risk_probability_matrix = pd.read_csv('deck_risk_probability_matrix.csv')
oe_constraint_probability_matrix = pd.read_csv('oe_constraint_probability_matrix.csv')
ppap_risk_probability_matrix = pd.read_csv('ppap_risk_probability_matrix.csv')
stop_auto_buy_probability_matrix = pd.read_csv('stop_auto_buy_probability_matrix.csv')
rev_stop_probability_matrix = pd.read_csv('rev_stop_probability_matrix.csv')

In [13]:
def predict(x):   
    if type(x) == dict:
        dataframe = pd.DataFrame(x, index=[0], columns=['sku', 'national_inv', 'lead_time', 'in_transit_qty',
                                                    'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
                                                    'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
                                                    'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
                                                    'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
                                                    'ppap_risk', 'stop_auto_buy', 'rev_stop'])
    else:
        dataframe = x
    
    dataframe = dataframe.drop('sku', axis=1) #dropping sku column
    
    if dataframe.iloc[-1].isna().all() == True:
        dataframe = dataframe[:-1] #removing last row as there are NaN values
    
    dataframe = dataframe.fillna(lead_time_mean) #mean imputation
    dataframe.replace({'Yes': 1, 'No': 0}, inplace=True) #converting categorical features into binary features
    
    #adding binary_pieces_past_due
    conditions = [dataframe['pieces_past_due'] == 0, dataframe['pieces_past_due'] > 0]
    values = [0, 1]
    dataframe['binary_pieces_past_due'] = np.select(conditions, values)
    
    #adding binary_local_bo_qty
    conditions = [dataframe['local_bo_qty'] == 0, dataframe['local_bo_qty'] > 0]
    values = [0, 1]
    dataframe['binary_local_bo_qty'] = np.select(conditions, values)
    
    #imputing all categorical features
    conditions_pt = [dataframe['potential_issue'] == 0, dataframe['potential_issue'] == 1]
    values_pt = [potential_issue_probability_matrix['No'][0], potential_issue_probability_matrix['No'][1]]
    dataframe['potential_issue'] = np.select(conditions_pt, values_pt)

    conditions_dr = [dataframe['deck_risk'] == 0, dataframe['deck_risk'] == 1]
    values_dr = [deck_risk_probability_matrix['No'][0], deck_risk_probability_matrix['No'][1]]
    dataframe['deck_risk'] = np.select(conditions_dr, values_dr)

    conditions_oe = [dataframe['oe_constraint'] == 0, dataframe['oe_constraint'] == 1]
    values_oe = [oe_constraint_probability_matrix['No'][0], oe_constraint_probability_matrix['No'][1]]
    dataframe['oe_constraint'] = np.select(conditions_oe, values_oe)

    conditions_pp = [dataframe['ppap_risk'] == 0, dataframe['ppap_risk'] == 1]
    values_pp = [ppap_risk_probability_matrix['No'][0], ppap_risk_probability_matrix['No'][1]]
    dataframe['ppap_risk'] = np.select(conditions_pp, values_pp)

    conditions_stp = [dataframe['stop_auto_buy'] == 0, dataframe['stop_auto_buy'] == 1]
    values_stp = [stop_auto_buy_probability_matrix['No'][0], stop_auto_buy_probability_matrix['No'][1]]
    dataframe['stop_auto_buy'] = np.select(conditions_stp, values_stp)

    conditions_rev = [dataframe['rev_stop'] == 0, dataframe['rev_stop'] == 1]
    values_rev = [rev_stop_probability_matrix['No'][0], rev_stop_probability_matrix['No'][1]]
    dataframe['rev_stop'] = np.select(conditions_rev, values_rev)

    filename = 'best_model_forest.h5'
    best_model = pickle.load(open(filename, 'rb'))
    predictions = best_model.predict(dataframe)
    if len(predictions) == 1:
        predictions = int(predictions)
    return predictions    

In [35]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import accuracy_score, roc_curve, auc

#constants calculated from eda & feature engineering
lead_time_mean = float(np.load('lead_time_mean.npy'))
potential_issue_probability_matrix = pd.read_csv('potential_issue_probability_matrix.csv')
deck_risk_probability_matrix = pd.read_csv('deck_risk_probability_matrix.csv')
oe_constraint_probability_matrix = pd.read_csv('oe_constraint_probability_matrix.csv')
ppap_risk_probability_matrix = pd.read_csv('ppap_risk_probability_matrix.csv')
stop_auto_buy_probability_matrix = pd.read_csv('stop_auto_buy_probability_matrix.csv')
rev_stop_probability_matrix = pd.read_csv('rev_stop_probability_matrix.csv')

def predict(x):   
    if type(x) == dict:
        dataframe = pd.DataFrame(x, index=[0], columns=['sku', 'national_inv', 'lead_time', 'in_transit_qty',
                                                    'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
                                                    'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
                                                    'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
                                                    'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
                                                    'ppap_risk', 'stop_auto_buy', 'rev_stop'])
    else:
        dataframe = x
    
    dataframe = dataframe.drop('sku', axis=1) #dropping sku column
    
    if dataframe.iloc[-1].isna().all() == True:
        dataframe = dataframe[:-1] #removing last row as there are NaN values

    dataframe = dataframe.fillna(lead_time_mean) #mean imputation
    dataframe.replace({'Yes': 1, 'No': 0}, inplace=True) #converting categorical features into binary features
    
    #adding binary_pieces_past_due
    conditions = [dataframe['pieces_past_due'] == 0, dataframe['pieces_past_due'] > 0]
    values = [0, 1]
    dataframe['binary_pieces_past_due'] = np.select(conditions, values)
    
    #adding binary_local_bo_qty
    conditions = [dataframe['local_bo_qty'] == 0, dataframe['local_bo_qty'] > 0]
    values = [0, 1]
    dataframe['binary_local_bo_qty'] = np.select(conditions, values)
    
    #imputing all categorical features
    conditions_pt = [dataframe['potential_issue'] == 0, dataframe['potential_issue'] == 1]
    values_pt = [potential_issue_probability_matrix['No'][0], potential_issue_probability_matrix['No'][1]]
    dataframe['potential_issue'] = np.select(conditions_pt, values_pt)

    conditions_dr = [dataframe['deck_risk'] == 0, dataframe['deck_risk'] == 1]
    values_dr = [deck_risk_probability_matrix['No'][0], deck_risk_probability_matrix['No'][1]]
    dataframe['deck_risk'] = np.select(conditions_dr, values_dr)

    conditions_oe = [dataframe['oe_constraint'] == 0, dataframe['oe_constraint'] == 1]
    values_oe = [oe_constraint_probability_matrix['No'][0], oe_constraint_probability_matrix['No'][1]]
    dataframe['oe_constraint'] = np.select(conditions_oe, values_oe)

    conditions_pp = [dataframe['ppap_risk'] == 0, dataframe['ppap_risk'] == 1]
    values_pp = [ppap_risk_probability_matrix['No'][0], ppap_risk_probability_matrix['No'][1]]
    dataframe['ppap_risk'] = np.select(conditions_pp, values_pp)

    conditions_stp = [dataframe['stop_auto_buy'] == 0, dataframe['stop_auto_buy'] == 1]
    values_stp = [stop_auto_buy_probability_matrix['No'][0], stop_auto_buy_probability_matrix['No'][1]]
    dataframe['stop_auto_buy'] = np.select(conditions_stp, values_stp)

    conditions_rev = [dataframe['rev_stop'] == 0, dataframe['rev_stop'] == 1]
    values_rev = [rev_stop_probability_matrix['No'][0], rev_stop_probability_matrix['No'][1]]
    dataframe['rev_stop'] = np.select(conditions_rev, values_rev)

    filename = 'best_model_forest.h5'
    best_model = pickle.load(open(filename, 'rb'))
    predictions = best_model.predict(dataframe)
    if len(predictions) == 1:
        predictions = int(predictions)
    return predictions

UsageError: Line magic function `%` not found.


In [18]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from streamlit.proto.Markdown_pb2 import Markdown
import streamlit.components.v1 as components

In [39]:
import inference
from inference import predict

ImportError: cannot import name 'predict' from 'inference' (D:\Program files\Anaconda3\lib\site-packages\inference\__init__.py)

In [19]:
st.title('Prediction of Backorders in Inventory Management')
st.header('A Random Forest model trained with a balanced subsample class weight')
st.markdown('created by: **Pratyush Mohit**')
st.subheader("Upload a csv file")

2022-10-10 22:35:04.911 
  command:

    streamlit run D:\Program files\Anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [20]:
uploaded_file = st.file_uploader("Choose a file...", type=['csv'])
if uploaded_file is not None:
   dataframe = pd.read_csv(uploaded_file)
   st.write("Loading...displaying first five rows")
   st.dataframe(data=dataframe.head(), width=730, height=200)
   st.write("Predicting...")
   x = dataframe.drop('went_on_backorder', axis=1)
   fig, ax = plt.subplots()
   sns.heatmap(x.corr(), ax=ax)
   plt.title('Correlation Matrix')
   st.write(fig)
   predictions = predict(x)
   st.write(predictions)
   st.write('Done')

SyntaxError: invalid syntax (Temp/ipykernel_19552/570703633.py, line 1)

SyntaxError: invalid syntax (Temp/ipykernel_19552/722235828.py, line 1)